In [1]:
#Importamos modulos
import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
import timeit
from datetime import datetime
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
#from google.colab import files

C:\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Obtención de datos de fuentes seleccionadas

Datos de densidad de población

https://www.ine.es/covid/densidad_poblacion.zip

## Datos de movilidad de Google

https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=04188f017409e90a

In [2]:
url="https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=04188f017409e90a"
s=requests.get(url).content
raw_google=pd.read_csv(io.StringIO(s.decode('utf-8')))

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_google['country_region_code'].nunique()

134

In [4]:
raw_google.loc[(raw_google['country_region']=='Spain')]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
746284,ES,Spain,NaN,NaN,NaN,NaN,NaN,2020-02-15,2.0,-1.0,26.0,8.0,0.0,-2.0
746285,ES,Spain,NaN,NaN,NaN,NaN,NaN,2020-02-16,2.0,3.0,13.0,5.0,-1.0,-2.0
746286,ES,Spain,NaN,NaN,NaN,NaN,NaN,2020-02-17,0.0,1.0,9.0,3.0,3.0,-1.0
746287,ES,Spain,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,0.0,5.0,4.0,3.0,0.0
746288,ES,Spain,NaN,NaN,NaN,NaN,NaN,2020-02-19,0.0,1.0,11.0,2.0,3.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758438,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-21,-26.0,-11.0,65.0,-45.0,-48.0,7.0
758439,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-22,-36.0,-21.0,57.0,-46.0,-15.0,2.0
758440,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-23,-31.0,4.0,12.0,-44.0,7.0,-2.0
758441,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-24,-12.0,-5.0,69.0,-39.0,-45.0,7.0


Conjunto de datos para España (a nivel global)

In [5]:
df_Spain=raw_google.loc[(raw_google['country_region']=='Spain') & (raw_google['sub_region_1'].isna() ==True) & (raw_google['sub_region_2'].isna() ==True)]
df_Spain.shape

(193, 14)

In [6]:
df_Spain['date'].nunique()

193

Conjunto de datos de las comunidades autónomas

In [7]:
df_ca=raw_google.loc[(raw_google['country_region']=='Spain') & (raw_google['sub_region_1'].isna() ==False) & (raw_google['sub_region_2'].isna() ==True)]
df_ca.shape

(3667, 14)

In [8]:
df_ca['sub_region_1'].nunique()

19

Conjunto de datos de las provicias

In [9]:
df_provincias=raw_google.loc[(raw_google['country_region']=='Spain') & (raw_google['sub_region_1'].isna() ==False) & (raw_google['sub_region_2'].isna() ==False)]
df_provincias.shape

(8299, 14)

In [10]:
df_provincias

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
746670,ES,Spain,Andalusia,Almería,NaN,ES-AL,NaN,2020-02-15,5.0,-3.0,40.0,10.0,1.0,-2.0
746671,ES,Spain,Andalusia,Almería,NaN,ES-AL,NaN,2020-02-16,-2.0,0.0,-2.0,1.0,1.0,-1.0
746672,ES,Spain,Andalusia,Almería,NaN,ES-AL,NaN,2020-02-17,0.0,-2.0,3.0,5.0,3.0,-1.0
746673,ES,Spain,Andalusia,Almería,NaN,ES-AL,NaN,2020-02-18,-3.0,-3.0,-2.0,5.0,3.0,0.0
746674,ES,Spain,Andalusia,Almería,NaN,ES-AL,NaN,2020-02-19,-1.0,-3.0,3.0,4.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758438,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-21,-26.0,-11.0,65.0,-45.0,-48.0,7.0
758439,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-22,-36.0,-21.0,57.0,-46.0,-15.0,2.0
758440,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-23,-31.0,4.0,12.0,-44.0,7.0,-2.0
758441,ES,Spain,Valencian Community,Valencia,NaN,ES-V,NaN,2020-08-24,-12.0,-5.0,69.0,-39.0,-45.0,7.0


Seleccionamos los datos relativos a cada comunidad autónoma

In [11]:
raw_google=raw_google.loc[(raw_google['country_region']=='Spain') & (raw_google['sub_region_1'].isna() ==False) & (raw_google['sub_region_2'].isna() ==True)]
raw_google.reset_index(drop=True,inplace=True)

In [12]:
#Almacenamos los datos seleccionados
raw_google.to_csv(r'../datasets/raw_data/mobility_google.csv')
raw_google

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,ES,Spain,Andalusia,NaN,NaN,ES-AN,NaN,2020-02-15,5.0,-1.0,33.0,15.0,0.0,-2.0
1,ES,Spain,Andalusia,NaN,NaN,ES-AN,NaN,2020-02-16,6.0,4.0,17.0,10.0,1.0,-4.0
2,ES,Spain,Andalusia,NaN,NaN,ES-AN,NaN,2020-02-17,0.0,0.0,10.0,8.0,4.0,-1.0
3,ES,Spain,Andalusia,NaN,NaN,ES-AN,NaN,2020-02-18,-1.0,0.0,6.0,7.0,4.0,-1.0
4,ES,Spain,Andalusia,NaN,NaN,ES-AN,NaN,2020-02-19,0.0,0.0,7.0,6.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,ES,Spain,Valencian Community,NaN,NaN,ES-VC,NaN,2020-08-21,-13.0,-2.0,105.0,-38.0,-44.0,6.0
3663,ES,Spain,Valencian Community,NaN,NaN,ES-VC,NaN,2020-08-22,-23.0,-11.0,103.0,-37.0,-11.0,1.0
3664,ES,Spain,Valencian Community,NaN,NaN,ES-VC,NaN,2020-08-23,-19.0,NaN,41.0,-35.0,9.0,-1.0
3665,ES,Spain,Valencian Community,NaN,NaN,ES-VC,NaN,2020-08-24,-1.0,3.0,108.0,-33.0,-40.0,6.0


In [13]:
raw_google['sub_region_1'].unique()

array(['Andalusia', 'Aragon', 'Asturias', 'Balearic Islands',
       'Basque Country', 'Canary Islands', 'Cantabria',
       'Castile and León', 'Castile-La Mancha', 'Catalonia', 'Ceuta',
       'Community of Madrid', 'Extremadura', 'Galicia', 'La Rioja',
       'Melilla', 'Navarre', 'Region of Murcia', 'Valencian Community'],
      dtype=object)

## Datos de casos de COVID-19 por Comunidades Autónomas

In [14]:
url="https://cnecovid.isciii.es/covid19/resources/datos_ccaas.csv"
s=requests.get(url).content
raw_covid=pd.read_csv(url, sep=',')
raw_covid.to_csv(r'../datasets/raw_data/casos_covid.csv')
raw_covid

,ccaa_iso,fecha,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida
0,AN,2020-01-31,0,0,0,0,0
1,AR,2020-01-31,0,0,0,0,0
2,AS,2020-01-31,0,0,0,0,0
3,CB,2020-01-31,0,0,0,0,0
4,CE,2020-01-31,0,0,0,0,0
...,...,...,...,...,...,...,...
3909,ML,2020-08-23,13,12,0,0,1
3910,NC,2020-08-23,70,70,0,0,0
3911,PV,2020-08-23,636,636,0,0,0
3912,RI,2020-08-23,63,63,0,0,0
